<a href="https://colab.research.google.com/github/FatemeZamanian/DeepLearning/blob/main/Flowers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot as plt
import numpy as np
import os
import datetime

In [13]:
dataset_path="/content/drive/MyDrive/Flowers"
width=height=224
batch_size=32
data_generator=ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    rotation_range=10,
    zoom_range=0.1,
    validation_split=0.1
)

In [27]:
train_data=data_generator.flow_from_directory(
    os.path.join(dataset_path,'Train'),
    target_size=(width,height),
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    subset='training',

)

val_data=data_generator.flow_from_directory(
    os.path.join(dataset_path,'Train'),
    target_size=(width,height),
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    subset='validation',

)

Found 986 images belonging to 17 classes.
Found 102 images belonging to 17 classes.


In [28]:
model_v=tf.keras.applications.VGG16(
    input_shape=(width,height,3),
    include_top=False,
    weights='imagenet'
)

In [33]:
for layer in model_v.layers:
  layer.trainable=False

In [34]:
model=tf.keras.Sequential([
        model_v,
        layers.Flatten(),
        layers.Dense(1024,activation='relu'),
        layers.Dense(128,activation='relu'),
        layers.Dense(17,activation='softmax'),
])

In [ ]:
!pip install wandb
import wandb
from wandb.keras import WandbCallback

In [ ]:
wandb.init(project="Flowers")
config = wandb.config
config.learning_rate = 0.001

In [37]:

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate= config.learning_rate ),
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=['accuracy'],)

In [38]:
model.fit(train_data,
          steps_per_epoch=train_data.samples//batch_size,
          validation_data=val_data,
          validation_steps=val_data.samples/batch_size,
          epochs=10,
          callbacks=[WandbCallback()]

    
)

Epoch 1/10
30/30 [==============================] - 24s 757ms/step - loss: 3.6342 - accuracy: 0.3124 - val_loss: 1.2880 - val_accuracy: 0.6078
Epoch 2/10
30/30 [==============================] - 22s 729ms/step - loss: 0.9026 - accuracy: 0.7306 - val_loss: 0.6655 - val_accuracy: 0.7941
Epoch 3/10
30/30 [==============================] - 22s 730ms/step - loss: 0.4567 - accuracy: 0.8753 - val_loss: 0.5619 - val_accuracy: 0.8039
Epoch 4/10
30/30 [==============================] - 22s 729ms/step - loss: 0.3022 - accuracy: 0.9208 - val_loss: 0.3136 - val_accuracy: 0.8922
Epoch 5/10
30/30 [==============================] - 22s 728ms/step - loss: 0.1464 - accuracy: 0.9748 - val_loss: 0.4216 - val_accuracy: 0.8431
Epoch 6/10
30/30 [==============================] - 22s 726ms/step - loss: 0.1195 - accuracy: 0.9811 - val_loss: 0.2792 - val_accuracy: 0.8922
Epoch 7/10
30/30 [==============================] - 22s 726ms/step - loss: 0.1042 - accuracy: 0.9832 - val_loss: 0.3416 - val_accuracy: 0.8922